In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 24 13:02:32 2021

@author: lenovo
"""
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
import math
import matplotlib.gridspec as gridspec
from plotting import newfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import layers, activations
from scipy.interpolate import griddata
from eager_lbfgs import lbfgs, Struct
from pyDOE import lhs

# from generate_dataset import *
# from utilities import *
import os.path

import shutil
import datetime
os.environ["KMP_WARNINGS"] = "FALSE" 

In [2]:
class neural_net(object):

    def __init__(self, layers):

        self.layer_sizes = layers



        # L-BFGS weight getting and setting from https://github.com/pierremtb/PINNs-TF2.0



    def xavier_init(self, layer_sizes):
        in_dim = layer_sizes[0]
        out_dim = layer_sizes[1]
        xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
        return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)

    def model(self):

        input_tensor = keras.Input(shape=(self.layer_sizes[0],))

        hide_layer_list = []
        flag = True
        for width in self.layer_sizes[1:-1]:
            if flag:
                x = layers.Dense(
                    width, activation=tf.nn.tanh,
                    kernel_initializer="glorot_normal")(input_tensor)
                flag = False
            else:
                x = layers.Dense(
                    width, activation=tf.nn.tanh,
                    kernel_initializer="glorot_normal")(x)
        output_tensor = layers.Dense(self.layer_sizes[-1], activation=None,kernel_initializer="glorot_normal")(x)
        # print("xxxxxxxxxxxxxx")
        output0 = output_tensor[:, 0:1]
        output1 = output_tensor[:, 1:2]
        output2 = output_tensor[:, 2:3]

        model_output = keras.models.Model(input_tensor, [output0, output1, output2])
        model_output.summary()

        return model_output



In [3]:
class TwoD_BFS_Slip_PINN:

    def __init__(self, data, layers  , activFun , mode , starter_learning_rate = 1.0e-3 , ExistModel=0):

        self.mode = mode

        self.dirname, logpath = self.make_output_dir()
        self.logger = self.get_logger(logpath)     
        self.layers = layers # 
        self.weight_initial = tf.Variable([1.0], dtype=tf.float32)
        self.weight_wall = tf.Variable([1.0], dtype=tf.float32)
        self.weight_inlet = tf.Variable([1.0], dtype=tf.float32)
        self.weight_outlet = tf.Variable([1.0], dtype=tf.float32)

        self.weight_fu = tf.Variable([1.0], dtype=tf.float32)

        self.optimizer = tf.keras.optimizers.Adam(lr=0.005, beta_1=.99)
        self.optimizer_fu= tf.keras.optimizers.Adam(lr=0.003, beta_1=.99)
        self.optimizer_initial = tf.keras.optimizers.Adam(lr=0.03, beta_1=.99)
        self.optimizer_wall = tf.keras.optimizers.Adam(lr=0.03, beta_1=.99)
        self.optimizer_inlet = tf.keras.optimizers.Adam(lr=0.03, beta_1=.99)
        self.optimizer_outlet = tf.keras.optimizers.Adam(lr=0.03, beta_1=.99)

        # initialize the NN
        self.net_cuvwp = neural_net(layers).model()

        # self.loss_tensor_list = [ self.loss_OUTLET,  self.loss_INLET , self.loss_WALL, self.loss_res, self.loss_INITIAL] 
        self.loss_list = ["loss_OUTLET" , "loss_INLET", "loss_WALL" , "loss_Phys", "loss_Initial" ]
        self.epoch_loss = dict.fromkeys(self.loss_list, 0)
        self.loss_history = dict((loss, []) for loss in self.loss_list)

        self.sizes_w = []
        self.sizes_b = []
        for i, width in enumerate(self.layer_sizes):
            if i != 1:
                self.sizes_w.append(int(width * self.layer_sizes[1]))
                self.sizes_b.append(int(width if i != 0 else self.layer_sizes[1]))

    def set_weights(self, w, sizes_b):  # 重新设置参数

        for i, layer in enumerate(self.net_cuvwp.layers[1:len(self.sizes_w) + 1]):
            start_weights = sum(self.sizes_w[:i]) + sum(sizes_b[:i])
            end_weights = sum(self.sizes_w[:i + 1]) + sum(sizes_b[:i])
            weights = w[start_weights:end_weights]
            w_div = int(self.sizes_w[i] / sizes_b[i])
            weights = tf.reshape(weights, [w_div, sizes_b[i]])
            biases = w[end_weights:end_weights + sizes_b[i]]
            weights_biases = [weights, biases]
            layer.set_weights(weights_biases)


    def get_weights(self):
        w = []
        for layer in self.net_cuvwp.layers[1:len(self.sizes_w) + 1]:
            weights_biases = layer.get_weights()
            weights = weights_biases[0].flatten()
            biases = weights_biases[1]
            w.extend(weights)
            w.extend(biases)
        w = tf.convert_to_tensor(w)
        return w
    
    def plot_loss_history(self , path):

        fig = plt.figure(4, figsize=(13, 4))
        loss_list = ["loss_OUTLET" , "loss_INLET", "loss_WALL" ,"loss_Initial"]
        ax = plt.subplot(1 , 2 , 1)
    #         fig.set_size_inches([15,8])
        for key in loss_list:
            self.print("Final loss %s: %e" % (key, self.loss_history[key][-1]))
            ax.semilogy(self.loss_history[key], label=key)
        ax.set_xlabel("epochs", fontsize=7)
        ax.set_ylabel("loss", fontsize=7)
        ax.set_yscale('log')
        ax.tick_params(labelsize=7)
        ax.legend()


        ax = plt.subplot(1,2,2)
        self.print("Final loss %s: %e" % ("loss_res", self.loss_history["loss_Phys"][-1]))
        ax.semilogy(self.loss_history["loss_res"], label="loss_Phys")
        ax.set_xlabel("epochs", fontsize=7)
        ax.set_ylabel("loss", fontsize=7)
        ax.set_yscale('log')
        ax.tick_params(labelsize=7)
        ax.legend()
        plt.savefig(path , bbox_inches='tight')
        plt.close("all")

    @tf.function
    def net_f(self,t ,  x, y):

        mu = 0.00345
        density = 1056

        u, v, p = self.net_cuvwp(tf.concat([t, x , y],1))

        
        u_t = tf.gradients(u, t)[0]
        u_x = tf.gradients(u, x)[0]
        u_y = tf.gradients(u, y)[0]
        u_xx = tf.gradients(u_x, x)[0]
        u_yy = tf.gradients(u_y, y)[0]

        v_t = tf.gradients(v, t)[0]
        v_x = tf.gradients(v, x)[0]
        v_y = tf.gradients(v, y)[0]
        v_xx = tf.gradients(v_x, x)[0]
        v_yy = tf.gradients(v_y, y)[0]

        p_x = tf.gradients(p, x)[0]
        p_y = tf.gradients(p, y)[0]

        f_u = u_t + (u * u_x + v * u_y ) + 1.0/density * p_x - mu *(u_xx + u_yy )
        f_v = v_t + (u * v_x + v * v_y ) + 1.0/density * p_y - mu *(v_xx + v_yy )
        div = u_x + v_y 
            

        return f_u, f_v, div


    # define the loss
    def loss_funct(self ,t_c_tf , x_c_tf, y_c_tf, t_WALL_tf , x_WALL_tf, y_WALL_tf, p_WALL_tf ,
                    t_INLET_tf , x_INLET_tf, y_INLET_tf, u_INLET_tf ,  v_INLET_tf ,t_OUTLET_tf , x_OUTLET_tf, y_OUTLET_tf, u_OUTLET_tf , v_OUTLET_tf,
                    t_INITIAL_tf , x_INITIAL_tf, y_INITIAL_tf , u_INITIAL_tf, v_INITIAL_tf, p_INITIAL_tf ):


        [f_u, f_v, f_e] = self.net_f(t_c_tf ,x_c_tf ,y_c_tf)
    
        [u_WALL_pred, v_WALL_pred, p_WALL_pred] = self.net_cuvwp(tf.concat([t_WALL_tf, x_WALL_tf,y_WALL_tf],1))
        
        [u_INLET_pred, v_INLET_pred, p_INLET_pred] = self.net_cuvwp(tf.concat([t_INLET_tf, x_INLET_tf,y_INLET_tf],1))

        [u_OUTLET_pred, v_OUTLET_pred,  p_OUTLET_pred ]= self.net_cuvwp(tf.concat([t_OUTLET_tf,x_OUTLET_tf, y_OUTLET_tf],1))

        
        [u_INITIAL_pred, v_INITIAL_pred,  p_INITIAL_pred ]= self.net_cuvwp( tf.concat([t_INITIAL_tf, x_INITIAL_tf, y_INITIAL_tf  ],1))

        loss_res = self.weight_fu *( tf.reduce_mean(tf.square(f_u)) + 1.0 *  tf.reduce_mean(tf.square(f_v)) + tf.reduce_mean(tf.square(f_e)))
        
        ##############################################################################
        # loss_res = 1.0 *tf.reduce_mean(tf.square(u_c_pred  - u_c_tf)) + 1 * tf.reduce_mean(tf.square(v_c_pred  -v_c_tf )) + 1 * tf.reduce_mean(tf.square(p_c_pred - p_c_tf)) 
    
    # on the Wall :supervised leaning:  U_predict - Uexact , V_predict - V_exact and  P_predict - P_exact and the gradient : P_y = 0  
        loss_wall =   self.weight_wall *  (tf.reduce_mean(tf.square(u_WALL_pred ))  + 1.0* tf.reduce_mean(tf.square(v_WALL_pred)))
                                                                                                        # + 1000 * tf.reduce_mean(tf.square(p_WALL_pred - p_WALL_tf)) + tf.square(tf.gradients(p_WALL_pred, y_WALL_tf)[0])) 
        
        
    # on the INLET :supervised leaning:  U_predict - Uexact , V_predict - V_exact and  P_predict - P_exact and the gradient : P_x = 0  
        loss_inlet =   self.weight_inlet  * ( tf.reduce_mean(tf.square(u_INLET_pred - u_INLET_tf))  +  1.0 * tf.reduce_mean(tf.square(v_INLET_pred -v_INLET_tf )))
        #  + \                1000.0 * tf.reduce_mean(tf.square(p_INLET_pred - p_INLET_tf)) #+ tf.square(tf.gradients(p_INLET_pred, x_INLET_tf)[0])) 
        
    # on the OUTLET :supervised leaning:  U_predict - Uexact , V_predict - V_exact and  P_predict - P_exact and the gradient : U_x = 0 and V_x = 0   
        loss_outlet =   self.weight_outlet * (tf.reduce_mean(tf.square(p_OUTLET_pred )) )# 50.0  *    tf.reduce_mean(tf.square(u_OUTLET_pred - u_OUTLET_tf) + tf.square(tf.gradients(u_OUTLET_pred , x_OUTLET_tf)[0])) + \        2000.0 * tf.reduce_mean(tf.square(v_OUTLET_pred - v_OUTLET_tf) + tf.square(tf.gradients(v_OUTLET_pred , x_OUTLET_tf)[0])) +  
        

    # on the Initial condition :supervised leaning:  U_predict - Uexact , V_predict - V_exact and  P_predict - P_exact 
        loss_initial = self.weight_initial *  (tf.reduce_mean(tf.square(u_INITIAL_pred - u_INITIAL_tf)) +  tf.reduce_mean(tf.square(v_INITIAL_pred - v_INITIAL_tf)) +  tf.reduce_mean(tf.square(p_INITIAL_pred - p_INITIAL_tf)))

        
        loss = loss_wall + loss_initial + loss_res + loss_inlet + loss_outlet

        return  loss_res, loss_wall , loss_inlet , loss_outlet , loss_initial , loss
    


    @tf.function
    def grad(self ,t_c_tf , x_c_tf, y_c_tf,
             t_WALL_tf , x_WALL_tf, y_WALL_tf, p_WALL_tf ,
               t_INLET_tf , x_INLET_tf, y_INLET_tf, u_INLET_tf ,  v_INLET_tf ,
                 t_OUTLET_tf , x_OUTLET_tf, y_OUTLET_tf, u_OUTLET_tf , v_OUTLET_tf,
                 t_INITIAL_tf , x_INITIAL_tf, y_INITIAL_tf , u_INITIAL_tf, v_INITIAL_tf, p_INITIAL_tf ):

        with tf.GradientTape(persistent=True) as tape:

            loss_res, loss_wall , loss_inlet , loss_outlet , loss_initial , loss = self.loss_funct(t_c_tf , x_c_tf, y_c_tf, t_WALL_tf , x_WALL_tf, y_WALL_tf, p_WALL_tf ,
                                                                                                   t_INLET_tf , x_INLET_tf, y_INLET_tf, u_INLET_tf ,  v_INLET_tf ,t_OUTLET_tf , x_OUTLET_tf, y_OUTLET_tf, u_OUTLET_tf , v_OUTLET_tf,
                                                                                                     t_INITIAL_tf , x_INITIAL_tf, y_INITIAL_tf , u_INITIAL_tf, v_INITIAL_tf, p_INITIAL_tf )
            grads = tape.gradient(loss, self.net_cuvwp.trainable_variables)

            grads_initial = tape.gradient(loss, self.weight_initial)
            grads_fu = tape.gradient(loss, self.weight_fu)

            grads_wall = tape.gradient(loss, self.weight_wall)
            grads_inlet = tape.gradient(loss, self.weight_inlet)

            grads_outlet = tape.gradient(loss, self.weight_outlet)


        return   loss_res, loss_wall , loss_inlet , loss_outlet , loss_initial , loss, grads , grads_fu , grads_initial, grads_wall , grads_inlet , grads_outlet



#model.train(collo[::10,:], inlet, outlet, wall, initial,  weight_ub, weight_fu, tf_iter=1000,   tf_iter2=100, newton_iter2=1500)
    def train( self , collo, inlet, outlet, wall, initial ,  tf_iter, tf_iter2, newton_iter2):

        batch_size =  120
        n_batches =  collo.shape[0] // batch_size
        start_time = time.time()

        running_time = 0


        # tf.print(f"weight_initial: {self.weight_initial}  weight_fu: {self.weight_fu}")
        print("starting Adam training")

        a = np.random.rand(1000)
        loss_history = list(a)
        MSE_b0 = list(a)
        MSE_f0 = list(a)

        MSE_b1 = []
        MSE_f1 = []
        # print(wall)
        # For mini-batch (if used)
        for epoch in range(tf_iter):
            for it in range(n_batches):

                t_WALL_tf= wall[: , 0:1]
                x_WALL_tf= wall[: , 1:2]
                y_WALL_tf= wall[: , 2:3]
                p_WALL_tf= wall[: , 5:6]

                t_INLET_tf= inlet[: , 0:1]
                x_INLET_tf= inlet[: , 1:2]
                y_INLET_tf= inlet[: , 2:3]
                u_INLET_tf= inlet[: , 3:4]
                v_INLET_tf= inlet[: , 4:5]
                p_INLET_tf= inlet[: , 5:6]

                t_OUTLET_tf= outlet[: , 0:1]
                x_OUTLET_tf= outlet[: , 1:2]
                y_OUTLET_tf= outlet[: , 2:3]
                u_OUTLET_tf= outlet[: , 3:4]
                v_OUTLET_tf= outlet[: , 4:5]

                t_INITIAL_tf= initial[: , 0:1]
                x_INITIAL_tf= initial[: , 1:2]
                y_INITIAL_tf= initial[: , 2:3]
                u_INITIAL_tf= initial[: , 3:4]
                v_INITIAL_tf= initial[: , 4:5]
                p_INITIAL_tf= initial[: , 5:6]


                x_f_tf = collo[:, 1:2][it * batch_size:(it * batch_size + batch_size), ]
                y_f_tf = collo[:, 2:3][it * batch_size:(it * batch_size + batch_size), ]
                t_f_tf = collo[:, 0:1][it * batch_size:(it * batch_size + batch_size), ]
                
                elapsed = time.time() - start_time
                running_time += elapsed

                loss_res, loss_wall , loss_inlet , loss_outlet , loss_initial , loss,grads , grads_fu , grads_initial, grads_wall , grads_inlet , grads_outlet = self.grad( t_f_tf , x_f_tf, y_f_tf,
                                                                                                                                                         t_WALL_tf , x_WALL_tf, y_WALL_tf, p_WALL_tf ,
                                                                                                                                                           t_INLET_tf , x_INLET_tf, y_INLET_tf, u_INLET_tf ,  v_INLET_tf ,
                                                                                                                                                             t_OUTLET_tf , x_OUTLET_tf, y_OUTLET_tf, u_OUTLET_tf , v_OUTLET_tf,
                                                                                                                                                               t_INITIAL_tf , x_INITIAL_tf, y_INITIAL_tf , u_INITIAL_tf, v_INITIAL_tf, p_INITIAL_tf )
                
                

                # print("starting Adam training")
        # self.loss_list = ["loss_OUTLET" , "loss_INLET", "loss_WALL" , "loss_Phys", "loss_Initial" ]

                self.optimizer.apply_gradients(zip(grads, self.net_cuvwp.trainable_variables))


                batch_losses =[loss_outlet.numpy()[0], loss_inlet.numpy()[0] , loss_wall.numpy()[0] , loss_res.numpy()[0] ,loss_initial.numpy()[0] ]

                self.assign_batch_losses(batch_losses)
                for key in self.loss_history:
                    self.loss_history[key].append(self.epoch_loss[key])
                start_time = time.time()
                                    
                # if loss_history[-1] < loss_history[-2] and loss_history[-2] < loss_history[-3] and loss_history[-1] < loss_history[-10]:
                self.optimizer_outlet.apply_gradients(zip([-grads_outlet], [self.weight_outlet]))
                self.optimizer_inlet.apply_gradients(zip([-grads_inlet], [self.weight_inlet]))
                self.optimizer_wall.apply_gradients(zip([-grads_wall], [self.weight_wall]))
                self.optimizer_initial.apply_gradients(zip([-grads_initial], [self.weight_initial]))
                self.optimizer_fu.apply_gradients(zip([-grads_fu], [self.weight_fu]))

            if epoch % 10 == 0:
                elapsed = time.time() - start_time
                self.print('Epoch: %d| It: %d| Loss: %.3e| physLoss: %.3e | lINITIAL: %.3e | wLoss: %.3e | inLoss: %.3e | outLoss: %.3e |Time: %.2fs | rTime: %.3eh | LR: %.3e'
                          %(epoch , it, loss.numpy()[0], loss_res.numpy()[0], loss_initial.numpy()[0] , loss_wall.numpy()[0] , loss_inlet.numpy()[0] , loss_outlet.numpy()[0] , elapsed, running_time, 0.005))

                # wu = self.weight_initial.numpy()
                # wf = self.weight_fu.numpy()
  

            #l-bfgs-b optimization
            print("Starting L-BFGS training")

            loss_and_flat_grad = self.get_loss_and_flat_grad(t_f_tf , x_f_tf, y_f_tf,t_WALL_tf , x_WALL_tf, y_WALL_tf, p_WALL_tf ,
                                                              t_INLET_tf , x_INLET_tf, y_INLET_tf, u_INLET_tf ,  v_INLET_tf ,
                                                                  t_OUTLET_tf , x_OUTLET_tf, y_OUTLET_tf, u_OUTLET_tf , v_OUTLET_tf,
                                                                    t_INITIAL_tf , x_INITIAL_tf, y_INITIAL_tf , u_INITIAL_tf, v_INITIAL_tf, p_INITIAL_tf)

            lbfgs(loss_and_flat_grad, self.get_weights(self.net_cuvwp), Struct(), maxIter=100, learningRate=0.8)


        return #MSE_b1, MSE_f1,  weightu, weightf

    def assign_batch_losses(self, batch_losses):
        for loss_values, key in zip(batch_losses, self.epoch_loss):
            self.epoch_loss[key] = loss_values
            
            

    # L-BFGS implementation from https://github.com/pierremtb/PINNs-TF2.0
    def get_loss_and_flat_grad ( self , t_c_tf , x_c_tf, y_c_tf, t_WALL_tf , x_WALL_tf, y_WALL_tf, p_WALL_tf , 
                                t_INLET_tf , x_INLET_tf, y_INLET_tf, u_INLET_tf ,  v_INLET_tf ,
                                t_OUTLET_tf , x_OUTLET_tf, y_OUTLET_tf, u_OUTLET_tf , v_OUTLET_tf,
                                  t_INITIAL_tf , x_INITIAL_tf, y_INITIAL_tf , u_INITIAL_tf, v_INITIAL_tf, p_INITIAL_tf):
        
        def loss_and_flat_grad(w):
            with tf.GradientTape() as tape:
                self.net_cuvwp.set_weights( w, self.net_cuvwp.sizes_w, self.net_cuvwp.sizes_b)
                loss_res, loss_wall , loss_inlet , loss_outlet , loss_initial , loss = self.loss_funct(t_c_tf , x_c_tf, y_c_tf, t_WALL_tf , x_WALL_tf, y_WALL_tf, p_WALL_tf ,
                    t_INLET_tf , x_INLET_tf, y_INLET_tf, u_INLET_tf ,  v_INLET_tf ,t_OUTLET_tf , x_OUTLET_tf, y_OUTLET_tf, u_OUTLET_tf , v_OUTLET_tf,
                    t_INITIAL_tf , x_INITIAL_tf, y_INITIAL_tf , u_INITIAL_tf, v_INITIAL_tf, p_INITIAL_tf)
                
            grad = tape.gradient(loss, self.net_cuvwp.trainable_variables)
            grad_flat = []
            for g in grad:
                grad_flat.append(tf.reshape(g, [-1]))
            grad_flat = tf.concat(grad_flat, 0)
            # print(loss_value, grad_flat)
            return loss, grad_flat

        return loss_and_flat_grad


    def predict(self , t , x , y):
        X_star = tf.concat([t,x, y],1) # tf.convert_to_tensor(np.array([t , x, y]), dtype=tf.float32)

        u, v, p = self.net_cuvwp(tf.concat([ X_star[:, 0:1] , X_star[:, 1:2], X_star[:, 2:3]], 1))

        return u.numpy(), v.numpy(), p.numpy()

############################################################
   ############################################################
    def make_output_dir(self):
        
        if not os.path.exists("checkpoints"):
            os.mkdir("checkpoints")
        text = datetime.now().strftime("%b-%d-%Y_%H-%M-%S-%f_") + self.mode
        dirname = os.path.abspath(os.path.join("checkpoints", text))
        os.mkdir(dirname)
        text = 'output.log'
        logpath = os.path.join(dirname, text)
        shutil.copyfile('dwPINNS.ipynb', os.path.join(dirname, 'dwPINNS.ipynb'))

        return dirname, logpath
    

    #  ############################################################
    # def make_output_dir(self):
        
    #     if not os.path.exists("/okyanus/users/afarea/PINN/Adaptive_PINN/noslipwall_02_Z0slice_2D/checkpoints"):
    #         os.mkdir("/okyanus/users/afarea/PINN/Adaptive_PINN/noslipwall_02_Z0slice_2D/checkpoints")
    #     dirname = os.path.join("/okyanus/users/afarea/PINN/Adaptive_PINN/noslipwall_02_Z0slice_2D/checkpoints", datetime.now().strftime("%b-%d-%Y_%H-%M-%S-%f_") + self.mode)
    #     os.mkdir(dirname)
    #     text = 'output.log'
    #     logpath = os.path.join(dirname, text)
    #     shutil.copyfile('/okyanus/users/afarea/PINN/Adaptive_PINN/noslipwall_02_Z0slice_2D/M1.py', os.path.join(dirname, 'M1.py'))

    #     return dirname, logpath
    

    def get_logger(self, logpath):
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.DEBUG)
        sh = logging.StreamHandler()
        sh.setLevel(logging.DEBUG)        
        sh.setFormatter(logging.Formatter('%(message)s'))
        fh = logging.FileHandler(logpath)
        logger.addHandler(sh)
        logger.addHandler(fh)
        return logger
    
    def print(self, *args):
        for word in args:
            if len(args) == 1:
                self.logger.info(word)
            elif word != args[-1]:
                for handler in self.logger.handlers:
                    handler.terminator = ""
                if type(word) == float or type(word) == np.float64 or type(word) == np.float32: 
                    self.logger.info("%.4e" % (word))
                else:
                    self.logger.info(word)
            else:
                for handler in self.logger.handlers:
                    handler.terminator = "\n"
                if type(word) == float or type(word) == np.float64 or type(word) == np.float32:
                    self.logger.info("%.4e" % (word))
                else:
                    self.logger.info(word)
                    

In [4]:
########################################################################################

def get_training_dataset(path , pINLET , pOUTLET , pWALL , pDomain , pInitial , dist):
    
    data = h5py.File(path , 'r')  # load dataset from matlab
    WALL = np.transpose(data['noslipwall_02_Z0slice_2D_wall'], axes=range(len(data['noslipwall_02_Z0slice_2D_wall'].shape) - 1,-1, -1)).astype(np.float32)
    
    WALL = np.delete(WALL , np.where(WALL[:,0] == WALL[:,0].min())[0] , 0)  
    # WALL = np.delete(WALL , np.where(WALL[:,1] <= 0.2)[0] , 0)  


    domain = np.transpose(data['noslipwall_02_Z0slice_2D_innerdomain'], axes=range(len(data['noslipwall_02_Z0slice_2D_innerdomain'].shape) - 1,-1, -1)).astype(np.float32)
    
    domain = np.delete(domain , np.where(domain[:,0] == domain[:,0].min())[0] , 0)  
    # domain = np.delete(domain , np.where(domain[:,1] <= 0.2)[0] , 0)  


    # INLET = np.transpose(data['noslipwall_02_Z0slice_2D_inlet'],  axes=range(len(data['noslipwall_02_Z0slice_2D_inlet'].shape) - 1,-1, -1)).astype(np.float32)
        
    INLET = domain[np.where(domain[:,1] == domain[:,1].min())[0],:] #np.delete(INLET , np.where(INLET[:,0] == INLET[:,0].min())[0] , 0)  

    OUTLET = np.transpose(data['noslipwall_02_Z0slice_2D_outlet'], axes=range(len(data['noslipwall_02_Z0slice_2D_outlet'].shape) - 1,-1, -1)).astype(np.float32)
    
    OUTLET = np.delete(OUTLET , np.where(OUTLET[:,0] == OUTLET[:,0].min())[0] , 0)  

    total = INLET.shape[0] + OUTLET.shape[0] + domain.shape[0] +  WALL.shape[0] 
    
    np.random.seed(1234)

    # initial domain ux is 0.2 and other values are zero. FOr wall, all values are zero
    
    INITIALd = domain[np.where(domain[:,0] == domain[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALw = WALL[np.where(WALL[:,0] == WALL[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALi = INLET[np.where(INLET[:,0] == INLET[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALo = OUTLET[np.where(OUTLET[:,0] == OUTLET[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    #random selection of training data
    INITIAL = np.concatenate([INITIALd , INITIALo],0)
    
    # domain = np.concatenate([domain , WALL , INLET , OUTLET],0)

    #INLET = np.delete(INLET , idx_initi , 0)  
    #OUTLET = np.delete(OUTLET , idx_inito  , 0)  
    #domain = np.delete(domain , idx_initd , 0)  
    #WALL = np.delete(WALL , idx_initw , 0)  
    
    if dist == "Sobol":
        idxi = generate_sobol_sequence(0 , INLET.shape[0] ,  int(INLET.shape[0] * pINLET)) 
        INLET = INLET[idxi, :]
        idxi = generate_sobol_sequence(0 , OUTLET.shape[0] ,  int(OUTLET.shape[0] * pOUTLET)) 
        OUTLET = OUTLET[idxi, :]
        idxi = generate_sobol_sequence(0 , INITIAL.shape[0] ,  int(INITIAL.shape[0] * pInitial)) 
        INITIAL = INITIAL[idxi, :]
        idxi = generate_sobol_sequence(0 , WALL.shape[0] ,  int(WALL.shape[0] * pWALL)) 
        WALL = WALL[idxi, :]
        idxi = generate_sobol_sequence(0 , domain.shape[0] ,  int(domain.shape[0] * pDomain)) 
        domain = domain[idxi, :]
    else:
        idxi = np.random.choice(INLET.shape[0], int(INLET.shape[0] * pINLET), replace=False)
        INLET = INLET[idxi, :]
        idxi = np.random.choice(OUTLET.shape[0], int(OUTLET.shape[0] * pOUTLET), replace=False)
        OUTLET = OUTLET[idxi, :]
        idxi = np.random.choice(INITIAL.shape[0], int(INITIAL.shape[0] * pInitial), replace=False)
        INITIAL = INITIAL[idxi, :]
        idxi = np.random.choice(WALL.shape[0], int(WALL.shape[0] * pWALL), replace=False)
        WALL = WALL[idxi, :]
        idxi = np.random.choice(domain.shape[0], int(domain.shape[0] * pDomain), replace=False)
        domain = domain[idxi, :]

    #########################################
    return [domain , INLET , OUTLET, WALL, INITIAL , total]
################################################################



def plot_result2(model ,path ):


    ######################################
    # model.save_NN()

    # list_ = [ "noslipwall_02_Z0slice_2D_wall", "noslipwall_02_Z0slice_2D_innerdomain" ]
    # N_data = [ 1600 , 20800 ]

    # peotDic = dict((key,value) for key,value in zip(list_,N_data))

    # tstep = [ 100 , 100 , ]
    # tstepList = dict((key,value) for key,value in zip(list_,tstep))

    # for key,value in peotDic.items():
    #     # print(key, value)
    #     test_data_inlet = get_testing_dataset(path   , part=key)
    #     test_data_inlet = test_data_inlet[np.argsort(test_data_inlet[:, 0])]

    #     [_ ,xf , _ , ufa , vfa  , pfa, u_pred , v_pred, p_pred]  = predict_result(model , test_data_inlet ,  value ,  tstep  ,  text=key , stm = False)
    #     peotDic[key] =  error_over_time(tstepList[key] ,value , ufa , vfa , pfa , u_pred , v_pred , p_pred )
    #     l1l2Erorr(key , u_pred ,v_pred , p_pred , ufa , vfa , pfa , model)

    # draw_error_over_time(peotDic , model.dirname)

    #     ## drawing velocity profile
    # xValues = [5.000e-04 , 0.2275, 9.995e-01]
    # # UVelocity_profile(model.dirname , 0.2 , xValues , xf , u_pred)

    
    tstep =100
    N_data = 20800
    #def predict_result(model , test_data , N_data , tstep  , text = 'all'  , stm = False):
    part = 'noslipwall_02_Z0slice_2D_innerdomain'
    test_data_innerdomain = get_testing_dataset(path    ,  part=part)
    [tf , xf , yf , ufa , vfa  , pfa, u_pred , v_pred, p_pred]   = predict_result(model , test_data_innerdomain , N_data ,  tstep ,  text='domain' ,  stm = False)

    

    x = xf.reshape(tstep,N_data)[0,:]
    y = yf.reshape(tstep,N_data)[0,:]
    t = tf.reshape(tstep,N_data)[:,0].T
    ufa = ufa.reshape(tstep,N_data)
    vfa = vfa.reshape(tstep,N_data)
    pfa = pfa.reshape(tstep,N_data)
    u_pred = u_pred.reshape(tstep,N_data)
    v_pred = v_pred.reshape(tstep,N_data)
    p_pred = p_pred.reshape(tstep,N_data)
    error_u =  np.abs(ufa - u_pred) # (u - u_pred) / u # np.abs(u - u_pred)
    error_v =  np.abs(vfa - v_pred) # (v - v_pred) / v # np.abs(v - v_pred)
    error_p =  np.abs(pfa - p_pred) # (p - p_pred) / p # np.abs(p - p_pred)

    data = [u_pred, v_pred , p_pred , ufa, vfa , pfa , error_u ,  error_v ,error_p ]

    plot_time_profile(model.dirname , x , y , t , ufa , u_pred , '$u$')
    plot_time_profile(model.dirname , x , y , t , vfa , v_pred , '$v$')
    plot_time_profile(model.dirname , x , y , t , pfa , p_pred , '$p$')

    draw_contourf(t , x , y , data , 1.0 ,10 , model.dirname , 5 , fontsize=17.5 , labelsize=12.5 , axes_pad=1.2)

    # model.weight_change_per_layer()



In [5]:
from generate_dataset import * 
# path = '/okyanus/users/afarea/dataDir/noslipwall_02_Z0slice_2D/noslipwall_02_Z0slice_2D.mat'
path = '/media/afrah2/MyWork/files2023/dataset/noslipwall_02_Z0slice_2D.mat'

modelPath = ''


pINLET = 1
pOUTLET= 0.3
pWALL =  0.006
pDomain = 0.0007
pInitial = 0.07

dist =  "Sobol"
activFun = 'hard_swish'

[collo , inlet , outlet, wall, initial , total] = get_training_dataset(path , pINLET , pOUTLET , pWALL , pDomain , pInitial , dist)

XY_c = np.concatenate([collo , wall, inlet , outlet], 0) 



# # Define model
mode = 'M1'

input_dimension = 3
output_dimension = 3
n_hidden_layers = 3
neurons = 20 # [3, 20, 20, 20, 20, 20, 20, 20, 3]

starter_learning_rate = float(0.005)
epochs = 5
model_layers = [input_dimension] + n_hidden_layers*[neurons] + [output_dimension]

batch_size= 300

iterations = 40000

method  = "mini_batch"


###############################################################
model = TwoD_BFS_Slip_PINN(XY_c , model_layers  , activFun , mode , starter_learning_rate , ExistModel = modelPath)


# model.print("Using mode: " , model.mode)
# model.print("neural network: " , model.layers )
# model.print("collocation Training data size : " ,collo.shape[0], " (" ,str(pDomain*100) , "%)")
# model.print("Wall Training data size: " ,wall.shape[0], " (" ,str(pWALL*100) , "%)")
# model.print("Initial Training data size: " ,initial.shape[0], " (" ,str(pInitial*100) , "%)")
# model.print("INLET Training data size: " ,inlet.shape[0], " (" ,str(pINLET*100) , "%)")
# model.print("OUTLET Training data size: " ,outlet.shape[0], " (" ,str(pOUTLET*100) , "%)")

# model.print("Total training datset size: " ,XY_c.shape[0], " (" ,str((XY_c.shape[0] / total)*100) , "%)")
# model.print("Activation function: " , activFun)
# model.print("number of iterations: " , iterations)

# model.print("Method desciption : learning rates: Exponential decay  with initial value: ", starter_learning_rate)


# model.print("File directory: " , model.dirname)

# #save_data_to_matlab(os.path.join(model.dirname,'slipwall_02_Z0slice_2D_training.mat') ,coll , INLET , OUTLET , WALL ,INITIAL )

# plot_dataset( model.dirname , wall , inlet , outlet , collo , initial , dist)




xxxxxxxxxxxxxx
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 20)           80          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 20)           420         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 20)           420         dense_1[0][0]                    
_______________________________________________________________________________

In [6]:
N_f = 10000
Nu1 = 200



model.train(collo[::10,:], inlet, outlet, wall, initial, tf_iter=1000,   tf_iter2=100, newton_iter2=1500)




starting Adam training


Epoch: 0| It: 0| Loss: 8.700e-01| physLoss: 6.468e-02 | lINITIAL: 6.771e-02 | wLoss: 3.937e-01 | inLoss: 3.187e-01 | outLoss: 2.522e-02 |Time: 0.05s | rTime: 6.022e-04h | LR: 5.000e-03
Epoch: 10| It: 0| Loss: 2.293e-01| physLoss: 7.472e-03 | lINITIAL: 3.361e-02 | wLoss: 1.224e-01 | inLoss: 6.054e-02 | outLoss: 5.326e-03 |Time: 0.02s | rTime: 3.872e-01h | LR: 5.000e-03
Epoch: 20| It: 0| Loss: 1.312e-01| physLoss: 4.908e-03 | lINITIAL: 1.759e-02 | wLoss: 4.401e-02 | inLoss: 4.903e-02 | outLoss: 1.568e-02 |Time: 0.02s | rTime: 6.787e-01h | LR: 5.000e-03
Epoch: 30| It: 0| Loss: 1.538e-01| physLoss: 9.331e-03 | lINITIAL: 2.990e-02 | wLoss: 2.264e-02 | inLoss: 2.613e-02 | outLoss: 6.577e-02 |Time: 0.02s | rTime: 8.770e-01h | LR: 5.000e-03
Epoch: 40| It: 0| Loss: 9.283e-02| physLoss: 7.845e-03 | lINITIAL: 1.422e-02 | wLoss: 3.296e-02 | inLoss: 4.336e-03 | outLoss: 3.347e-02 |Time: 0.02s | rTime: 1.172e+00h | LR: 5.000e-03
Epoch: 50| It: 0| Loss: 8.841e-02| physLoss: 1.714e-02 | lINITIAL: 1.56

In [7]:
key = "noslipwall_02_Z0slice_2D_innerdomain" 

test_data_innerdomain = get_testing_dataset(path   , part=key)


In [8]:
plot_result2(model , path)


Predicting nn solution

 Relative L2 ERROR:
domain U velocity  : 9.547e+01 %
domain V velocity  : 1.966e+04 %
domain P Pressure  : 9.417e+01 %

 Relative l1 error
domain U velocity  : 9.287e+01 %
domain V velocity  : 3.356e+04 %
domain P Pressure  : 8.756e+01 %
/media/afrah2/MyWork/files2023/SK/dwPINNs/utilities.py:653: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/media/afrah2/MyWork/files2023/SK/dwPINNs/utilities.py:653: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/media/afrah2/MyWork/files2023/SK/dwPINNs/utilities.py:653: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


AttributeError: 'TwoD_BFS_Slip_PINN' object has no attribute 'weight_change_per_layer'

In [1]:
model.weight_initial

NameError: name 'model' is not defined

In [10]:
model.plot_loss_history(model.dirname)


Final loss loss_OUTLET: 8.999521e-05
Final loss loss_INLET: 9.842000e-03
Final loss loss_WALL: 5.892984e-02
Final loss loss_Initial: 1.546057e-02


KeyError: 'loss_res'

In [10]:
test_data_innerdomain[:, 1:2].shape

test = ([test_data_innerdomain[:, 1:2] , test_data_innerdomain[:, 2:3] , test_data_innerdomain[:, 0:1]])
len(test)

3

In [11]:
# tf.print(f"weight_ub: {weight_initial}  weight_fu: {weight_fu}")
u_pred, v_pred, p_pred = model.predict(test_data_innerdomain[:,  0:1] , test_data_innerdomain[:,  1:2]  , test_data_innerdomain[:,  2:3])



# lbfgs(loss_and_flat_grad, get_weights(u_model), Struct(), maxIter=newton_iter2, learningRate=0.8)

In [12]:
error_u = np.linalg.norm(test_data_innerdomain[:,  3:4] - u_pred, 2) / np.linalg.norm(test_data_innerdomain[:,  3:4] , 2)
print('Error u: %e' % (error_u))
error_v = np.linalg.norm(test_data_innerdomain[:,  4:5] - v_pred, 2) / np.linalg.norm(test_data_innerdomain[:,  4:5], 2)
print('Error v: %e' % (error_v))
error_v = np.linalg.norm(test_data_innerdomain[:,  5:6] - v_pred, 2) / np.linalg.norm(test_data_innerdomain[:,  5:6], 2)
print('Error v: %e' % (error_v))
print("Starting L-BFGS training")


Error u: 8.883702e-01
Error v: 2.271106e+02
Error v: 1.168959e+00
Starting L-BFGS training


In [34]:
loss_and_flat_grad = get_loss_and_flat_grad(t_star1 , x_star1, y_star1, tb ,  xb, yb,  ub,  vb , weight_initial, weight_fu)

lbfgs(loss_and_flat_grad,  get_weights(u_model), Struct(), maxIter = 500, learningRate=0.8)

u_pred, v_pred, p_pred = predict(test_data_innerdomain[:,  0:3])
error_u = np.linalg.norm(u_exact1 - u_pred, 2) / np.linalg.norm(u_exact1, 2)
print('Error u: %e' % (error_u))
error_v = np.linalg.norm(v_exact1 - v_pred, 2) / np.linalg.norm(v_exact1, 2)
print('Error v: %e' % (error_v))

error_p = np.linalg.norm(p_exact1 - p_pred, 2) / np.linalg.norm(p_exact1, 2)
print('Error p: %e' % (error_p))


TypeError: loss_funct() missing 1 required positional argument: 'weight_fu'

In [16]:

elapsed = time.time() - start_time
print('Training time: %.4f' % (elapsed))

u_pred, v_pred, p_pred = predict(X_star1)

# U_pred = u_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
# V_pred = v_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
# P_pred = p_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
u_exact1 = np.array([X_star1[:, 3]])
v_exact1 = np.array([X_star1[:, 4]])
p_exact1 = np.array([X_star1[:, 5]])


error_uu = np.abs( u_exact1 - u_pred)
error_vv = np.abs(v_exact1 - v_pred)
error_pp = np.abs( p_exact1 - p_pred)

error_u = np.linalg.norm(u_exact1 - u_pred, 2) / np.linalg.norm(u_exact1, 2)
print('Error u: %e' % (error_u))

error_v = np.linalg.norm(v_exact1 - v_pred, 2) / np.linalg.norm(v_exact1, 2)
print('Error v: %e' % (error_v))

error_p = np.linalg.norm(p_exact1 - p_pred, 2) / np.linalg.norm(p_exact1, 2)
print('Error p: %e' % (error_p))

# dataNewNS = 'NS_hisyory.mat'
# scipy.io.savemat(dataNewNS, {'w_MSE_b': MSE_b1, 'w_MSE_f': MSE_f1, 'weight_u': weightu,
#                   'weight_f': weightf, 'U_pred': u_pred, 'V_pred': V_pred, 'P_pred': P_pred})

Training time: 1488.1751
Error u: 4.266133e+01
Error v: 4.266161e+01
Error p: 8.245512e+01
